In [4]:
import data
import models
import cache
import seaborn as sns
import numpy as np
import pandas as pd
import patsy
from matplotlib import pyplot as plt

In [5]:
sns.set(context='talk')

In [6]:
model_name = 'model5.3.1'
by = 'SubSet'
sample_n = 500
nu = 2

## sample data for analysis

In [7]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_500.pkl
INFO:stancache.stancache:prep_sample_df: Loading result from cache


## fit model

In [8]:
stan_data = models.prep_stan_data(sample_df, by=by, nu=nu)

In [17]:
model_file = models.get_model_file(model_name=model_name)
print(cache._read_file(model_file))

## neg binom parameterization
## estimate correlation matrix among cell types
functions {
  /* 
    ## per : https://groups.google.com/forum/?fromgroups#!searchin/stan-users/neg_binomial_2_log_rng/stan-users/rgpHgBtFS3k/A7GqglKOAQAJ
    ## neg_binomial_2_rng(mu,psi) as suggested by Ben: 
    real gamma_temp; 
    if (is_inf(psi)) 
      gamma_temp <- mu; 
    else 
      gamma_temp <- gamma_rng(psi, psi / mu); 
    if (gamma_temp < pow(2.0, 30.0)) 
      result <- poisson_rng(gamma_temp); 
    else 
      result <- normal_rng(gamma_temp, sqrt(gamma_temp)); 
  */
  real alt_neg_binom_2_rng(real mu, real psi) {
    real gamma_temp; 
    real result;
    
    if (is_nan(psi)) 
      gamma_temp = mu;
    else if (is_inf(psi)) 
      gamma_temp = mu; 
    else if (is_nan(psi / mu))
      gamma_temp = mu;
    else if (is_inf(psi / mu))
      gamma_temp = mu;
    else if (psi / mu < 0)
      gamma_temp = mu;
    else 
      gamma_temp = gamma_rng(psi, psi / mu);
      
    if (is_nan(gamma_te

In [ ]:
model_fit = models.cached_stan_fit(file=model_file, data=stan_data, model_name=model_name)

## check convergence (superficially)

In [ ]:
models.print_stan_summary(model_fit, pars='lp__')

In [ ]:
models.plot_stan_summary(model_fit, pars='theta', metric='Rhat')

## summarize posterior draws of theta by gene

In [ ]:
# meta-data used for plotting functions below
# so that the following code is invariant to the model run
colnames = list(stan_data['x'].columns)
sort_by = colnames[0]
print(sort_by)

In [ ]:
theta_ldf = models.prep_theta_summary(model_fit, sample_df=sample_df, colnames=colnames, expose_group=sort_by)

In [ ]:
## show theta estimates for first 50 genes, by `sort-by`
g = sns.boxplot(data=theta_ldf.loc[theta_ldf['mean_value_rank_{}'.format(sort_by)] <= 50,:] \
                .sort_values('mean_value_rank_{}'.format(sort_by)),
            y='new_gene_cat',
            x='value',
            hue='variable', 
            fliersize=0, width=2, linewidth=0.2)

In [ ]:
## zoom in on the highest-ranked genes by `sort-by` difference from average 
## across all cell types
g = sns.boxplot(data=theta_ldf.loc[theta_ldf['mean_abs_diff_rank_{}'.format(sort_by)] <= 10,:] \
                .sort_values('mean_diff_rank_{}'.format(sort_by)),
            y='new_gene_cat',
            x='value',
            hue='variable', 
            fliersize=0, linewidth=0.2)

## posterior-predictive checking for selected genes

In [ ]:
# identify top_genes by name
top_genes = theta_ldf.loc[theta_ldf['mean_abs_diff_rank_{}'.format(sort_by)] <= 10,:] \
                .drop_duplicates(subset='new_gene_cat')['new_gene_cat'].values
print(top_genes)

In [ ]:
# get yrep draws
yrep_df = models.prep_yrep_summary(model_fit, sample_df=sample_df, filter_genes=top_genes[0:3])

In [ ]:
# plot estimates & observed values for top 3 genes, by Subset
models.plot_posterior_predictive_checks(model_fit=model_fit, plot_genes=top_genes, sample_df=sample_df,
                                        yrep_df=yrep_df, n_genes=2)


## summarize posterior draws for `Omega`

In [ ]:
omega_summary = models.prep_omega_summary(stan_fit=model_fit, stan_data=stan_data, par='Omega', gene_id=by)

In [ ]:
with sns.plotting_context('paper'):
    sns.heatmap(omega_summary.loc[:, list(stan_data['x'].columns)])